In [25]:
import pandas as pd
from datetime import datetime

In [2]:
# Load the data from the provided file
file_path = 'Data.xlsx'  # Replace with the correct path if needed

# Load Sheet1 (assumed to be raw survey data)
data_raw = pd.read_excel(file_path, sheet_name='Sheet1')

C:\Users\srika\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### 1. Data Preprocessing

#### 1.1 Remove duplicates

In [3]:
# Check for duplicates
duplicates_count = data_raw.duplicated().sum()
print(f"Number of duplicate rows before removal: {duplicates_count}")

Number of duplicate rows before removal: 43


In [4]:
# Remove duplicates
data_cleaned = data_raw.drop_duplicates()

In [5]:
# Check if duplicates were removed
duplicates_count_after = data_cleaned.duplicated().sum()
print(f"Number of duplicate rows after removal: {duplicates_count_after}")

Number of duplicate rows after removal: 0


#### 1.2 Handle missing values

In [6]:
# check missing values
data_cleaned.isna().sum()

Are you still going to school?                                                                                                                                                                                                                                 2
Do you have any other children living in your house with you?                                                                                                                                                                                                  2
How many people live in your home with you (including adults)?                                                                                                                                                                                                 2
What year are you in now?                                                                                                                                                                                                            

In [7]:
# Identify categorical and numerical columns
categorical_columns = data_cleaned.select_dtypes(include=['object', 'category']).columns
numerical_columns = data_cleaned.select_dtypes(include=['number']).columns

In [8]:
# Fill missing values for categorical columns with the mode
for col in categorical_columns:
    mode_value = data_cleaned[col].mode()[0]  # Get the mode (most frequent value)
    data_cleaned[col].fillna(mode_value, inplace=True)

C:\Users\srika\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [9]:
# Fill missing values for numerical columns with the median
for col in numerical_columns:
    median_value = data_cleaned[col].median()  # Get the median
    data_cleaned[col].fillna(median_value, inplace=True)

C:\Users\srika\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\srika\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\srika\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\srika\anac

In [10]:
# verify missing valued are fixed
data_cleaned.isna().sum()

Are you still going to school?                                                                                                                                                                                                                                0
Do you have any other children living in your house with you?                                                                                                                                                                                                 0
How many people live in your home with you (including adults)?                                                                                                                                                                                                0
What year are you in now?                                                                                                                                                                                                               

##### Interpretation:
* Categorical Variables: Missing values were replaced with the mode (most frequent value) of each column. This ensures that the missing values do not disrupt the distribution significantly.
* Numerical Variables: Missing values were replaced with the median, which is robust to outliers and preserves central tendency without being influenced by extreme values.
##### Result:
* Missing values before handling: 86
* Missing values after handling: 0
* Cleaned data is saved in Data_Handled_Missing.xlsx.

In [11]:
# Save the cleaned data
cleaned_file_path = 'Data_Handled_Missing.xlsx'  # Replace with the desired path
data_cleaned.to_excel(cleaned_file_path, index=False)

print(f"Cleaned data saved to: {cleaned_file_path}")

Cleaned data saved to: Data_Handled_Missing.xlsx


In [12]:
# Step 3: Drop unnecessary columns
columns_to_drop = ['ID', 'Year', 'Month', 'Day']  # Specify columns to drop
data_final = data_cleaned.drop(columns=columns_to_drop, errors='ignore')

In [13]:
# Step 4: Drop the additional column specified
column_to_drop = ['27. If yes, how are you keeping in touch (tick all you use)?']
data_final_dropped_column = data_final.drop(columns=column_to_drop, errors='ignore')

##### Drop Unnecessary Columns: 
* Columns ID, Year, Month, and Day were dropped as they are irrelevant for analysis.

In [17]:
data_final_dropped_column.rename(
    columns={'3. How many times did you brush your teeth YESTERDAY?.1': '4. What time did you fall asleep last night?'},
    inplace=True
)

In [23]:
# Save the final cleaned dataset
final_file_path = 'Data_Final_No_ID_Year_Month_Day.xlsx'  # Replace with desired path
data_final_dropped_column.to_excel(final_file_path, index=False)

print(f"Final cleaned data saved to: {final_file_path}")

Final cleaned data saved to: Data_Final_No_ID_Year_Month_Day.xlsx


In [30]:
Data_cleaned = pd.read_excel('Data_Final_No_ID_Year_Month_Day.xlsx')

In [31]:
# Normalize the time strings by ensuring a space before "AM/PM"
Data_cleaned['4. What time did you fall asleep last night?'] = Data_cleaned[
    '4. What time did you fall asleep last night?'
].str.replace(r'([apAP][mM])$', r' \1', regex=True)

Data_cleaned['5. What time did you wake up TODAY (to the nearest half hour)?'] = Data_cleaned[
    '5. What time did you wake up TODAY (to the nearest half hour)?'
].str.replace(r'([apAP][mM])$', r' \1', regex=True)

# Define the time format
time_format = "%I:%M %p"

# Calculate hours of sleep
Data_cleaned['hours of sleep had last night'] = Data_cleaned.apply(
    lambda row: (
        (datetime.strptime(row['5. What time did you wake up TODAY (to the nearest half hour)?'], time_format) -
         datetime.strptime(row['4. What time did you fall asleep last night?'], time_format)).seconds / 3600
    ) if pd.notnull(row['5. What time did you wake up TODAY (to the nearest half hour)?']) and
         pd.notnull(row['4. What time did you fall asleep last night?']) else None,
    axis=1
)

# Drop the original time columns
Data_cleaned.drop(
    columns=[
        '4. What time did you fall asleep last night?',
        '5. What time did you wake up TODAY (to the nearest half hour)?'
    ],
    inplace=True,
    errors='ignore'
)



Dataset saved with calculated hours of sleep to: Data_Reverted_Sleep_Calculation.xlsx


In [37]:
# Save the updated dataset
Data_cleaned.to_csv('Data_cleaned.csv', index=False)
print('Cleaned data saved')

Cleaned data saved


### Summary of Data cleaning

The dataset is now in a clean and standardized format, ready for Exploratory Data Analysis (EDA). Here's why:

* Duplicates Removed: All duplicate rows have been removed, ensuring unique records.
* Missing Values Handled: Missing values have been replaced appropriately:
    * Mode for categorical variables.
    * Median for numerical variables.
    * Unnecessary Columns Dropped: Columns irrelevant to the analysis (e.g., ID, Year, Month, Day) have been removed.
    * Column Names Corrected: Duplicate and erroneous column names have been resolved.
Calculated New Features:
* "Hours of sleep had last night" was added by processing two time-related columns.
Standardized Data: Time formats and other textual inconsistencies have been normalized.